In [ ]:
import os
import numpy as np
#import datetime
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import matplotlib
from matplotlib import pyplot as plt
#from nilearn import datasets
#from nilearn import plotting
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr as corr

import tensorflow_datasets as tfds
import tensorflow as tf
from keras import datasets, Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from sklearn.metrics import confusion_matrix
from google.colab import drive #

In [ ]:
! git clone https://github.com/fsegatz01/IANNwTF_Group16.git

%cd IANNwTF_Group16/project/library

Cloning into 'IANNwTF_Group16'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 304 (delta 14), reused 0 (delta 0), pack-reused 277
Receiving objects: 100% (304/304), 13.31 MiB | 18.07 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [ ]:
# our own libraries

import model_library as ml
import data_processing_library as dpl

In [ ]:
### CONNECTION TO GOOGLE DRIVE ###
drive.mount('/content/drive', force_remount=True)
os.chdir ("drive/MyDrive")


In [ ]:
'''
### CONNECTION TO GOOGLE DRIVE ###
drive.mount('/content/drive', force_remount=True)
os.chdir ("drive/MyDrive")


data_dir = '/content/drive/MyDrive/algonauts_2023_tutorial_data' 
parent_submission_dir = '/content/drive/MyDrive/algonauts_2023_challenge_submission' 
'''

In [ ]:
# save images as tensors with train_NSD_as_Tensors() and test_NSD_as_Tensors() (in DataProcessing_Library) 

In [ ]:
############# PREPROCESS  NSD ###############

# we here only use the img data from one person since training the Model on all subjects data takes very long
nsd_train_images = dpl.preprocess_NSD(np.load('/content/drive/MyDrive/FinalProject/NSD_Data/nsd_train-images_subj01.npy'))

'''

### PUT ALL TRAIN IMAGES FROM DIFFERENT SUBJ TOGETHER ###
nsd_train_images = np.load('/content/drive/MyDrive/FinalProject/NSD_Data/nsd_train-images_subj01.npy')

for i in range(2,9):
  nsd_train_images = np.concatenate((nsd_train_images, np.load('/content/drive/MyDrive/FinalProject/NSD_Data/nsd_train-images_subj0{}.npy'.format(i))), axis=0)

'''

In [ ]:
######## RELOAD OLD TRAINED MODELS ########


### cifar cnn
cnn_cifar_model = ml.ConvModel()

cnn_cifar_model.compile(loss='categorical_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        metrics=['categorical_accuracy'])

# Restore the weights
cnn_cifar_model.load_weights('/checkpoints/cnn_cifar_model')


##############################################

### eco cnn
cnn_eco_model = ml.ConvModel()

cnn_eco_model.compile(loss='categorical_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        metrics=['categorical_accuracy'])

# Restore the weights
cnn_eco_model.load_weights('/checkpoints/cnn_eco_model')


##############################################

### cifar resnet50
resnet_cifar_model = ml.ResNetModel()

resnet_cifar_model.compile(loss='categorical_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        metrics=['categorical_accuracy'])

# Restore the weights
resnet_cifar_model.load_weights('{checkpoints/resnet_cifar_model')


#############################################

### eco resnet50
resnet_eco_model = ml.ResNetModel()

resnet_eco_model.compile(loss='categorical_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        metrics=['categorical_accuracy'])

# Restore the weights
resnet_cifar_model.load_weights('{checkpoints/resnet_eco_model')

In [ ]:
def cnn_output_to_input(model, layer_number, data):

  ''' takes the output from the layer layer_number in model
  when runnindg data through'''
      
    get_layer_output = K.function([model.layers[0].input],
                                      [model.layers[layer_number].output])
    layer_output = get_layer_output([data])[0]
    hidden_layer_outputs_flat = tf.keras.layers.Flatten()(layer_output)

    return hidden_layer_outputs_flat


cnn_features = resnet_output_to_input(cnn_cifar_model, 3, nsd_train_images)


def resnet_output_to_input(model, layer_number, data):

    ''' takes the output from the layer layer_number in model
    when runnindg data through'''
      
    get_layer_output = K.function([model.resnet50.input],
                                      [model.resnet50.layers[layer_number].output])
    layer_output = get_layer_output([data])[0]
    hidden_layer_outputs_flat = tf.keras.layers.Flatten()(layer_output)

    return hidden_layer_outputs_flat


In [ ]:
# Fit linear regressions on the training data
linReg_lh = LinearRegression().fit(cnn_features, np.load('/content/drive/MyDrive/algonauts_2023_tutorial_data/subj01/training_split/training_fmri/lh_training_fmri.npy'))
linReg_rh = LinearRegression().fit(cnn_features, np.load('/content/drive/MyDrive/algonauts_2023_tutorial_data/subj01/training_split/training_fmri/rh_training_fmri.npy'))


In [ ]:
#data augmentation
#add slightly transformed/rotated copies of already included data
augmentation_model = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.1),
])

